# SR3 Reader

* Reads data from SR3 file (HDF).
* Return groups, wells, special and grid properties list.
* Return 2D and 3D Time vector.
* Return a 2D or 3D property.
* Save all data to a csv file.

In [2]:
import h5py
from datetime import datetime, timedelta
import re
import numpy as np

In [3]:
with h5py.File(r'..\sr3\base_case_3a.sr3', 'r') as file:
    print("Main Groups in the HDF file:")
    for dataset in file.keys():
        print(f'  {dataset}')

    print("\nAll Groups in the HDF file:")
    def get_type(name):
        print(f'   {name:}\t{type(file[name])}')
    file.visit(get_type)

Main Groups in the HDF file:
  General
  SpatialProperties
  Tables
  TimeSeries

All Groups in the HDF file:
   General	<class 'h5py._hl.group.Group'>
   General/ComponentTable	<class 'h5py._hl.dataset.Dataset'>
   General/EventTable	<class 'h5py._hl.dataset.Dataset'>
   General/HistoryTable	<class 'h5py._hl.dataset.Dataset'>
   General/MasterTimeTable	<class 'h5py._hl.dataset.Dataset'>
   General/NameRecordTable	<class 'h5py._hl.dataset.Dataset'>
   General/TableAssociations	<class 'h5py._hl.dataset.Dataset'>
   General/UnitConversionTable	<class 'h5py._hl.dataset.Dataset'>
   General/UnitsTable	<class 'h5py._hl.dataset.Dataset'>
   SpatialProperties	<class 'h5py._hl.group.Group'>
   SpatialProperties/000000	<class 'h5py._hl.group.Group'>
   SpatialProperties/000000/BKRGCL	<class 'h5py._hl.dataset.Dataset'>
   SpatialProperties/000000/BKRGRL	<class 'h5py._hl.dataset.Dataset'>
   SpatialProperties/000000/BKROCRW	<class 'h5py._hl.dataset.Dataset'>
   SpatialProperties/000000/BKROCW	<cl

In [4]:
f = h5py.File(r'..\sr3\base_case_3a.sr3', 'r')

## Master Data

In [5]:
dataset = f['General/MasterTimeTable']
day_list = {number:days for (number,days) in zip(dataset['Index'], dataset['Offset in days'])}

def parse_date(date):
    date_string = str(date)
    integer_part = int(date_string.split('.')[0])
    decimal_part = float("0." + date_string.split('.')[1])

    parsed_date = datetime.strptime(str(integer_part), "%Y%m%d")
    fraction_of_day = timedelta(days=decimal_part)
    return parsed_date + fraction_of_day

dataset = f['General/MasterTimeTable']
date_list = {number:parse_date(date) for (number,date) in zip(dataset['Index'], dataset['Date'])}

In [290]:
dataset = f['General/UnitsTable']
unit_list = {number:{'type':name.decode().lower(), 'internal':in_name.decode(), 'current':out_name.decode(), 'conversion':dict()} for (number,name,in_name,out_name) in zip(dataset['Index'],dataset['Dimensionality'],dataset['Internal Unit'],dataset['Output Unit'])}

dataset = f['General/UnitConversionTable']
for (number, name, gain, offset) in zip(dataset['Dimensionality'],dataset['Unit Name'],dataset['Gain'],dataset['Offset']):
    unit_list[number]['conversion'][name.decode()] = (1./gain, offset * (-1.))

for d in unit_list.values():
    if d['internal'] != d['current']:
        gain, offset = d['conversion'][d['internal']]
        for k in d['conversion']:
            g, o = d['conversion'][k]
            d['conversion'][k] = (g / gain, o - offset)

for unit in unit_list.values():
    if unit['internal'] not in unit['conversion']:
        unit['conversion'][unit['internal']] = (1.,0.)

In [7]:
dataset = f['General/ComponentTable']
component_list = {(number+1):name[0].decode() for (number,name) in enumerate(dataset[:])}

def replace_components_property_list(property_list):
    pattern = re.compile(r'\((\d+)\)')
    def replace(match):
        number = int(match.group(1))
        return f'({str(component_list.get(number, match.group(1)))})'
    return {pattern.sub(replace, k):v for k,v in property_list.items()}

In [246]:
dataset = f['General/NameRecordTable']
property_list = dict()
for (keyword,name,long_name,dimensionality) in zip(dataset['Keyword'],dataset['Name'],dataset['Long Name'],dataset['Dimensionality']):
    if keyword != '':
        keyword = keyword.decode()
        name = name.decode()
        long_name = long_name.decode()
        dimensionality = dimensionality.decode()
        if keyword[-2:] == '$C':
            for c in component_list.values():
                property_list[f'{keyword[:-2]}({c})'] = {'name':name.replace('$C', f' ({c})'), 'long name':long_name.replace('$C', f' ({c})'), 'dimensionality_string':dimensionality}
        else:
            property_list[keyword] = {'name':name, 'long name':long_name, 'dimensionality_string':dimensionality}

In [286]:
def _unit_from_dimensionality(dimensionality_string):
    if dimensionality_string == '':
        return ''
    unit = ''
    if dimensionality_string[0] == '-':
        unit = '1'
    d = ''
    for c in dimensionality_string:
        if c == '|':
            unit = unit + unit_list[int(d)]['current']
            d = ''
        elif c == '-':
            unit = unit + '/'
        else:
            d = d + c
    return unit

def _unit_conversion_from_dimensionality(dimensionality_string, is_delta=False):
    if dimensionality_string == '':
        return (1., 0.)
    gain = 1.
    offset = 0.
    inverse = False
    if dimensionality_string[0] == '-':
        inverse = True
    d = ''
    for c in dimensionality_string:
        if c == '|':
            unit = unit_list[int(d)]['current']
            gain_new, offset_new = unit_list[int(d)]['conversion'][unit]
            d = ''
            if inverse:
                gain = gain / gain_new
                offset = 0.
            else:
                gain = gain * gain_new
                if is_delta:
                    offset = 0.
                else:
                    offset = offset * gain_new + offset_new
        elif c == '-':
            inverse = True
        else:
            d = d + c
    return (gain, offset)

def _update_properties_units():
    for p in property_list:
        property_list[p]['conversion'] = _unit_conversion_from_dimensionality(property_list[p]['dimensionality_string'])
        property_list[p]['unit'] = _unit_from_dimensionality(property_list[p]['dimensionality_string'])

def _get_unit_type_number(dimensionality):
    if isinstance(dimensionality, str):
        d = [d for d in unit_list if unit_list[d]['type'] == dimensionality.lower()]
        if len(d) == 0:
            raise ValueError(f'{dimensionality} is not a valid unit type.')
        if len(d) > 1:
            raise ValueError(f'{dimensionality} found more than once. Check code.')
        return d[0]
    else:
        return dimensionality

def set_current_unit(dimensionality, unit):
    dimensionality = _get_unit_type_number(dimensionality)
    if unit not in unit_list[dimensionality]['conversion']:
        raise ValueError(f'{unit} is not a valid unit for {unit_list[dimensionality]['type']}.')
    unit_list[dimensionality]['current'] = unit
    _update_properties_units()

def get_current_units():
    for d in unit_list.values():
        print(f'{d["type"]}: {d["current"]}')

In [314]:
def _get_unit_numbers(unit):
    out = list()
    for u in unit_list:
        for c in unit_list[u]['conversion']:
            if unit == c:
                out.append(u)
    return out

def add_new_unit(old_unit, new_unit, gain, offset):
    unit_numbers = _get_unit_numbers(old_unit)
    if len(unit_numbers) == 0:
        raise ValueError(f'{old_unit} was not found in Units Table.')

    for u in unit_numbers:
        g, o = unit_list[u]['conversion'][old_unit]
        unit_list[u]['conversion'][new_unit] = (g * gain, o * gain + offset)

In [288]:
def describe_property(property_):
    return {'description': property_list[property_]['name'],
            'long description': property_list[property_]['long name'],
            'unit': property_list[property_]['unit']
            }

In [319]:
additional_units = [('m3', 'MMm3', (1.e-6, 0.0)),
                    ('bbl', 'MMbbl', (1.e-6, 0.0)),
                    ('kg/cm2', 'kgf/cm2', (1., 0.0)),
                    ]

for old, new, (gain, offset) in additional_units:
    add_new_unit(old, new, gain, offset)

set_current_unit('pressure','kgf/cm2')
get_current_units()

time: day
temperature: C
pressure: kgf/cm2
length: m
property volume: m3
permeability: md
mass: kg
molar mass: gmole
viscosity: cp
energy: J
well liquid volume: m3
well gas volume: m3
well rate time: day
interfacial tension: dyne/cm
electrical current: A
electrical power: J/day
electrical potential: V
electrical resistance: ohm
electrical conductivity: S/m
electrical energy: J
temperature difference: C
diffusion/dispersion coeff.: cm2/s
concentration: kg/m3
molar concentration: gmole/m3


### Property Formulas (todo)

* Basic
    * 1035: sum (2 parameters)
    * 1037: sum (3 parameters)

* Time derivatives
    * 1040: derivative (1 parameter)
    * 1045: monthly derivative (1 parameter)
    * 1046: quarterly derivative (1 parameter)
    * 1047: yearly derivative (1 parameter)
    * 1048: daily derivative (1 parameter)
    * 1049: weekly derivative (1 parameter)

* Percentage
    * 1055: A*100 (1 parameter)

* Division
    * 1060: division (2 parameters)
    * 1062: division * 100 (2 parameters) <= used with on-fraction

    * 1080: division (2 parameters)

* Previous month
    * 1121: monthly derivative of previous month (1 parameter)
    * 1122: yearly derivative of previous month (1 parameter)

* Grid properties(?)
    * 1130: A*grid height (1 parameter)
    * 1140: A/(B*C) ? (3 parameters)
    * 1160: A*B (2 parameters)
    * 1200: ternary (2 parameters)
    * 1210: sum over layers (1 parameter)

* Per sector
    * 1110: ??? (1 parameter) <= per sector
    * 1245: ??? (1 parameter) <= per sector
    * 1246: derivative of 1245 (1 parameter) 



## Common Routines

In [95]:
def is_iterable(obj):
    try:
        iter(obj)
        return True
    except TypeError:
        return False

## Time Series

In [141]:
_element = {'special':{'':0}}
_parent = dict()
_connection = dict()
_property = dict()
_timestep = dict()
_day = dict()
_date = dict()

def _return_dataset(element_type, dataset_string):
    el_type_string = element_type.upper()
    if element_type == 'special':
        el_type_string = el_type_string + ' HISTORY'
    else:
        el_type_string = el_type_string + 'S'
    s = f'TimeSeries/{el_type_string}/{dataset_string}'
    if s in f:
        return f[s]
    else:
        raise ValueError(f'Dataset {dataset_string} not found for {element_type}. TimeSeries/{el_type_string}/{dataset_string} does not exist.')
    
def get_elements(element_type):        
    if element_type not in _element:
        dataset = _return_dataset(element_type=element_type, dataset_string='Origins')
        _element[element_type] = {name.decode():number for (number,name) in enumerate(dataset[:]) if name.decode()!=''}
    return _element[element_type]

def get_properties(element_type):
    if element_type not in _property:
        dataset = _return_dataset(element_type=element_type, dataset_string='Variables')
        _property[element_type] = {name.decode():number for (number,name) in enumerate(dataset[:])}
        _property[element_type] = replace_components_property_list(_property[element_type])
    return _property[element_type]

def get_timesteps(element_type):
    if element_type not in _timestep:
        dataset = _return_dataset(element_type=element_type, dataset_string='Timesteps')
        _timestep[element_type] = dataset[:]
    return _timestep[element_type]

def get_days(element_type):
    if element_type not in _day:
        timesteps = get_timesteps(element_type=element_type)
        _day[element_type] = np.vectorize(lambda x: day_list[x])(timesteps)
    return _day[element_type]

def get_dates(element_type):
    if element_type not in _date:
        timesteps = get_timesteps(element_type=element_type)
        _date[element_type] = np.vectorize(lambda x: date_list[x])(timesteps)
    return _date[element_type]

def _get_parents(element_type):
    if element_type in ['well', 'group', 'layer']:
        dataset = _return_dataset(element_type=element_type, dataset_string=f'{element_type.capitalize()}Table')
        def _name(name, parent):
            if element_type == 'layer':
                return f'{parent.decode()}{{{name.decode()}}}'
            else:
                return name.decode()
        _parent[element_type] = {_name(name, parent):parent.decode() for (name,parent) in zip(dataset['Name'], dataset['Parent'])}
    else:
        _parent[element_type] = {name:'' for name in get_elements(element_type)}

def get_parent(element_type, element_name):
    if element_type not in _parent:
        _get_parents(element_type)
    return _parent[element_type][element_name]

def _get_connections(element_type):
    if element_type == 'layer':
        dataset = _return_dataset(element_type=element_type, dataset_string=f'{element_type.capitalize()}Table')
        def _name(name, parent):
            return f'{parent.decode()}{{{name.decode()}}}'
        _connection[element_type] = {_name(name, parent):connection for (name,parent,connection) in zip(dataset['Name'], dataset['Parent'], dataset['Connect To'])}
    else:
        _parent[element_type] = {name:'' for name in get_elements(element_type)}

def get_connection(element_type, element_name):
    if element_type not in _connection:
        _get_connections(element_type)
    return _connection[element_type][element_name]

def _concatenate_arrays(arr1, arr2):
    if arr1.ndim == 1:
        arr1 = arr1.reshape(-1,1)
    if arr2.ndim == 1:
        arr2 = arr2.reshape(-1,1)

    return np.hstack((arr1, arr2))

def get_data(element_type, property_name, element_name='', with_dates=False, with_days=False, with_timesteps=False):
    dataset = _return_dataset(element_type=element_type, dataset_string='Data')
    
    properties = get_properties(element_type)
    if isinstance(property_name, list):
        property_list = [properties[name] for name in property_name]
    else:
        property_list = properties[property_name]

    elements = get_elements(element_type)
    if isinstance(element_name, list):
        element_list = [elements[name] for name in element_name]
    else:
        element_list = elements[element_name]
    
    if isinstance(property_name, list) and isinstance(element_name, list):
        data = dataset[:,property_list,element_list[0]]
        for el in element_list[1:]:
            data = _concatenate_arrays(data, dataset[:,property_list,el])
    else:
        data = dataset[:,property_list,element_list]

    if with_dates:
        return _concatenate_arrays(get_dates(element_type=element_type), data)
    elif with_days:
        return _concatenate_arrays(get_days(element_type=element_type), data)
    elif with_timesteps:
        return _concatenate_arrays(get_timesteps(element_type=element_type), data)
    else:
        return data

In [156]:
for p in get_properties('well'):
    print(p + ': ' + describe_property(p)['description'] + ' [' + describe_property(p)['unit'] + ']')

WELLSTATE: Well state []
WELLOPMO: Well operating mode []
WHP: Well Head Pressure [kPa]
BHP: Well Bottom-hole Pressure [kPa]
BLOCKP: Well Block Pressure [kPa]
MOBWTDATP: Well Mobility-Weighted Datum Pressure [kPa]
BHTEMP: Well bottom hole temperature [C]
WHTEMP: Well-head temperature [C]
ONFRAC: On-time Fraction []
CGLIFT: Cumulative lift gas injected per well [m3]
OILVOLSC: Cumulative Oil SC [m3]
GASVOLSC: Cumulative Gas SC [m3]
WATVOLSC: Cumulative Water SC [m3]
INLVOLSC: Cumulative INL SC [m3]
WTGVOLSC: Cumulative WTG SC [m3]
OILVOLRC: Cumulative Oil RC [m3]
GASVOLRC: Cumulative Gas RC [m3]
WATVOLRC: Cumulative Water RC [m3]
BHFVOLRC: Cumulative Bottom Hole Fluid RC [m3]
WELLOTIME: Well Cumulative Open On-time [day]
OILRATSC: Oil Rate SC [m3/day]
GASRATSC: Gas Rate SC [m3/day]
WATRATSC: Water Rate SC [m3/day]
INLRATSC: INL Rate SC [m3/day]
WTGRATSC: WTG Rate SC [m3/day]
OILRATRC: Oil Rate RC [m3/day]
GASRATRC: Gas Rate RC [m3/day]
WATRATRC: Water Rate RC [m3/day]
BHFRATRC: Bottom Ho

In [161]:
unit_conversion(get_data('well','BHP','P11'),

array([60627.83967735, 58480.50693715, 58438.01272767, 57980.06815396,
       57983.51690498, 57709.73044181, 57726.16221407, 57537.39631307,
       57538.77380028, 57365.2280972 , 57323.70980719, 57284.01254463,
       58281.72465427, 59325.41276696, 58475.7722709 , 57192.28291651,
       57202.87429949, 57063.16042717, 57058.48674533, 56919.6846983 ,
       56928.87977172, 56819.73285163, 57788.6784767 , 58832.04182623,
       57975.84282618, 56736.12066622, 56712.77887977, 56557.57044081,
       56564.27547255, 56460.95621084, 57464.14511838, 60283.58277288,
       60284.04794554, 60826.5486919 , 60826.77800729, 61167.16798526,
       61167.31680225, 61421.61570383, 61421.73735561, 61618.7333819 ,
       61618.8137225 , 61774.03522198, 61774.08756886, 61899.25700243,
       61899.39611201, 62003.36191368, 62003.39657825, 62089.97698518,
       62090.00585056, 62162.11965276, 62162.17581243, 62222.17658422,
       62222.21878842, 62271.99259186, 62272.00918382, 62313.07290233,
      

In [ ]:
dataset = f['TimeSeries/WELLS/Origins']
_element['well'] = {name.decode():number for (number,name) in enumerate(dataset[:]) if name.decode()!=''}

dataset = f['TimeSeries/WELLS/WellTable']
_parent['well'] = {name.decode():parent.decode() for (name,parent) in zip(dataset['Name'], dataset['Parent'])}

dataset = f['TimeSeries/WELLS/Variables']
_property['well'] = {name.decode():number for (number,name) in enumerate(dataset[:])}
_property['well'] = replace_components_property_list(_property['well'])

dataset = f['TimeSeries/WELLS/Timesteps']
_timestep['well'] = {number:timestep for (number,timestep) in enumerate(dataset[:])}

_day['well'] = [day_list[ts] for ts in _timestep['well'].values()]
_date['well'] = [date_list[ts] for ts in _timestep['well'].values()]


In [ ]:
dataset = f['TimeSeries/GROUPS/Origins']
group_list = {name.decode():number for (number,name) in enumerate(dataset[:]) if name.decode()!=''}

dataset = f['TimeSeries/GROUPS/GroupTable']
group_parent = {name.decode():parent.decode() for (name,parent) in zip(dataset['Name'], dataset['Parent'])}

dataset = f['TimeSeries/GROUPS/Variables']
group_property_list = {name.decode():number for (number,name) in enumerate(dataset[:])}
group_property_list = replace_components_property_list(group_property_list)

dataset = f['TimeSeries/GROUPS/Timesteps']
group_timestep_list = {number:timestep for (number,timestep) in enumerate(dataset[:])}

group_day_list = [day_list[ts] for ts in group_timestep_list.values()]
group_date_list = [date_list[ts] for ts in group_timestep_list.values()]

def get_group_property(property_name, group_name):
    dataset = f['TimeSeries/GROUPS/Data']
    return dataset[:,group_property_list[property_name],group_list[group_name]]

In [ ]:
dataset = f['TimeSeries/LAYERS/Origins']
layer_list = {name.decode():number for (number,name) in enumerate(dataset[:]) if name.decode()!=''}

dataset = f['TimeSeries/LAYERS/LayerTable']
layer_parent = {f'{parent.decode()}{{{name.decode()}}}':parent.decode() for (name,parent) in zip(dataset['Name'], dataset['Parent'])}

dataset = f['TimeSeries/LAYERS/LayerTable']
layer_connection = {f'{parent.decode()}{{{name.decode()}}}':connection for (name,parent,connection) in zip(dataset['Name'], dataset['Parent'], dataset['Connect To'])}

dataset = f['TimeSeries/LAYERS/Variables']
layer_property_list = {name.decode():number for (number,name) in enumerate(dataset[:])}
layer_property_list = replace_components_property_list(layer_property_list)

dataset = f['TimeSeries/LAYERS/Timesteps']
layer_timestep_list = {number:timestep for (number,timestep) in enumerate(dataset[:])}

layer_day_list = [day_list[ts] for ts in layer_timestep_list.values()]
layer_date_list = [date_list[ts] for ts in layer_timestep_list.values()]

def get_layer_property(property_name, layer_name):
    dataset = f['TimeSeries/LAYERS/Data']
    return dataset[:,layer_property_list[property_name],layer_list[layer_name]]

In [ ]:
dataset = f['TimeSeries/SECTORS/Origins']
sector_list = {name.decode():number for (number,name) in enumerate(dataset[:]) if name.decode()!=''}

dataset = f['TimeSeries/SECTORS/Variables']
sector_property_list = {name.decode():number for (number,name) in enumerate(dataset[:])}
sector_property_list = replace_components_property_list(sector_property_list)

dataset = f['TimeSeries/SECTORS/Timesteps']
sector_timestep_list = {number:timestep for (number,timestep) in enumerate(dataset[:])}

sector_day_list = [day_list[ts] for ts in sector_timestep_list.values()]
sector_date_list = [date_list[ts] for ts in sector_timestep_list.values()]

def get_sector_property(property_name, sector_name):
    dataset = f['TimeSeries/SECTORS/Data']
    return dataset[:,sector_property_list[property_name],sector_list[sector_name]]

In [ ]:
dataset = f['TimeSeries/SPECIAL HISTORY/Variables']
special_property_list = {name.decode():number for (number,name) in enumerate(dataset[:])}

dataset = f['TimeSeries/SPECIAL HISTORY/Timesteps']
special_timestep_list = {number:timestep for (number,timestep) in enumerate(dataset[:])}
special_property_list = replace_components_property_list(special_property_list)

special_day_list = [day_list[ts] for ts in special_timestep_list.values()]
special_date_list = [date_list[ts] for ts in special_timestep_list.values()]

def get_special_property(property_name):
    dataset = f['TimeSeries/SPECIAL HISTORY/Data']
    return dataset[:,special_property_list[property_name],0]

## Grid Properties

In [ ]:
dataset = f['SpatialProperties']
grid_timestep_list = dict()
i = 0
for key in dataset.keys():
    sub_dataset = f[f"SpatialProperties/{key}"]
    if isinstance(sub_dataset, h5py._hl.group.Group):
        grid_timestep_list[i] = key
        i += 1
    
grid_day_list = [day_list[int(ts)] for ts in grid_timestep_list.values()]
grid_date_list = [date_list[int(ts)] for ts in grid_timestep_list.values()]

In [ ]:
dataset = f['SpatialProperties/000000/GRID']
ni = dataset['IGNTID'][0]
nj = dataset['IGNTJD'][0]
nk = dataset['IGNTKD'][0]

n_cells = ni*nj*nk
print(f'{ni}x{nj}x{nk} = {n_cells} cells')

47x39x291 = 533403 cells


In [ ]:
dataset = f['SpatialProperties/Statistics']
grid_property_list = {name.decode().replace('/','%2F'):{'min':min_, 'max':max_, 'timesteps':set()} for name,min_,max_ in zip(dataset['Keyword'],dataset['Min'],dataset['Max'])}

def _list_grid_properties(timestep, add_timestep=True):
    dataset = f[f'SpatialProperties/{timestep}']
    for key in dataset.keys():
        sub_dataset = f[f"SpatialProperties/{timestep}/{key}"]
        if isinstance(sub_dataset, h5py._hl.dataset.Dataset):
            if key in grid_property_list:
                if 'size' in grid_property_list[key]:
                    if grid_property_list[key]['size'] != sub_dataset.size:
                        raise ValueError(f'Inconsistent grid size for {key}.')
                else:
                    grid_property_list[key]['size'] = sub_dataset.size
                if add_timestep:
                    grid_property_list[key]['timesteps'].add(timestep)
            else:
                raise ValueError(f'{key} not listed previously!')

_list_grid_properties('000000/GRID', False)
for ts in grid_timestep_list.values():
    _list_grid_properties(ts)

In [ ]:
n_active_cells = grid_property_list['IPSTCS']['size']
print(f'Number of active cells: {n_active_cells}')

Number of active cells: 67241


In [ ]:
for i in [3,4,234,434,1023,3400,5000]:
    print(f['SpatialProperties/000000/GRID/NODES'][i])

786744.8332000001
7281231.6992999995
776189.1926
5259.2539
785783.4680000001


7277323.4571
5352.8584


In [ ]:
grid_property_list

{'ICSTBC': {'min': -2.0, 'max': 0.0, 'timesteps': set(), 'size': 533403},
 'ICSTCG': {'min': 0.0, 'max': 0.0, 'timesteps': set(), 'size': 533403},
 'ICSTGN': {'min': 1.0, 'max': 1.0, 'timesteps': set(), 'size': 533403},
 'ICSTPS': {'min': 0.0, 'max': 67241.0, 'timesteps': set(), 'size': 533403},
 'ICSTPB': {'min': 1.0, 'max': 533403.0, 'timesteps': set(), 'size': 533403},
 'IPSTCS': {'min': 373.0, 'max': 533075.0, 'timesteps': set(), 'size': 67241},
 'IPSTBT': {'min': 9.0, 'max': 9.0, 'timesteps': set(), 'size': 67241},
 'IPSTAC': {'min': 1.0, 'max': 1.0, 'timesteps': set(), 'size': 67241},
 'IGNTID': {'min': 47.0, 'max': 47.0, 'timesteps': set(), 'size': 1},
 'IGNTJD': {'min': 39.0, 'max': 39.0, 'timesteps': set(), 'size': 1},
 'IGNTKD': {'min': 291.0, 'max': 291.0, 'timesteps': set(), 'size': 1},
 'IGNTZA': {'min': 0.0, 'max': 0.0, 'timesteps': set(), 'size': 1},
 'IGNTGT': {'min': 12.0, 'max': 12.0, 'timesteps': set(), 'size': 1},
 'IGNTFR': {'min': 533404.0, 'max': 533404.0, 'times

In [ ]:
dataset = f['SpatialProperties/000000/GRID']
for key in dataset.keys():
    sub_dataset = f[f"SpatialProperties/000000/GRID/{key}"]
    size = '{:,}'.format(sub_dataset.size)
    print(f'  {key}: {size} -> {property_list[key]["long name"]}')

  BLOCKDEPTH: 533,403 -> Grid block depths for all layers
  BLOCKPVOL: 67,241 -> Block pore volume
  BLOCKS: 4,267,224 -> Grid Block connectivity of Node Based Corner Point Grid
  BLOCKSIZE: 1,600,209 -> Grid block sizes in three directions
  BVOL: 67,241 -> Gross Block Volume = dx*dy*dz
  ICNTDR: 176,572 -> Connection direction
  ICSTBC: 533,403 -> Complete storage to block type
  ICSTCG: 533,403 -> Complete storage to child grid
  ICSTGN: 533,403 -> Complete storage to grid number
  ICSTPB: 533,403 -> Complete storage to parent block
  ICSTPS: 533,403 -> Complete storage to packed storage
  ICTPS1: 176,572 -> Connection to lower packed storage
  ICTPS2: 176,572 -> Connection to upper packed storage
  IGNTFR: 1 -> Grid number to first fracture CS index
  IGNTGT: 1 -> Grid number to grid type
  IGNTID: 1 -> Grid number to no. of I direction blocks
  IGNTJD: 1 -> Grid number to no. of J direction blocks
  IGNTKD: 1 -> Grid number to no. of K direction blocks
  IGNTNC: 2 -> Grid number t

In [ ]:
{p:grid_property_list[p] for p in grid_property_list if len(grid_property_list[p]['timesteps'])==0}

{'ICSTBC': {'min': -2.0, 'max': 0.0, 'timesteps': set(), 'size': 533403},
 'ICSTCG': {'min': 0.0, 'max': 0.0, 'timesteps': set(), 'size': 533403},
 'ICSTGN': {'min': 1.0, 'max': 1.0, 'timesteps': set(), 'size': 533403},
 'ICSTPS': {'min': 0.0, 'max': 67241.0, 'timesteps': set(), 'size': 533403},
 'ICSTPB': {'min': 1.0, 'max': 533403.0, 'timesteps': set(), 'size': 533403},
 'IPSTCS': {'min': 373.0, 'max': 533075.0, 'timesteps': set(), 'size': 67241},
 'IPSTBT': {'min': 9.0, 'max': 9.0, 'timesteps': set(), 'size': 67241},
 'IPSTAC': {'min': 1.0, 'max': 1.0, 'timesteps': set(), 'size': 67241},
 'IGNTID': {'min': 47.0, 'max': 47.0, 'timesteps': set(), 'size': 1},
 'IGNTJD': {'min': 39.0, 'max': 39.0, 'timesteps': set(), 'size': 1},
 'IGNTKD': {'min': 291.0, 'max': 291.0, 'timesteps': set(), 'size': 1},
 'IGNTZA': {'min': 0.0, 'max': 0.0, 'timesteps': set(), 'size': 1},
 'IGNTGT': {'min': 12.0, 'max': 12.0, 'timesteps': set(), 'size': 1},
 'IGNTFR': {'min': 533404.0, 'max': 533404.0, 'times

## Get Coordinates

In [94]:
_CHUNK_SIZE = 1200

def expand_list(original_list, items=1):
    expanded_list = []
    for value in original_list:
        value_shift = (value-1)*items
        expanded_list.append(value_shift)
        for new_value in range(value_shift + 1, value_shift + items):
            expanded_list.append(new_value)
    return expanded_list

def get_dataset_data(dataset, values_list):
    if not is_iterable(values_list):
        return dataset[values_list]
    x_ordered = list(set(values_list)).copy()
    x_ordered.sort()
    sub_x_ordered = np.array_split(x_ordered, int(len(x_ordered)/_CHUNK_SIZE)+1)
    y_dict = dict()
    for x in sub_x_ordered:
        y = dataset[x]
        for x,y in zip(x, y):
            y_dict[x] = y        
    return  np.array([y_dict[x] for x in values_list])
    
def get_cel_number(i,j,k, can_be_iterable=True):
    if can_be_iterable and is_iterable(i):
        return [ii + ni*(jj-1 + (kk-1)*nj) for (ii,jj,kk) in zip(i,j,k)]
    return i + ni*(j-1 + (k-1)*nj)

def get_nodes_index(i,j=None,k=None, can_be_iterable=True):
    dataset = f['SpatialProperties/000000/GRID/BLOCKS']
    if can_be_iterable and is_iterable(i):
        if j is not None:
            i = get_cel_number(i,j,k)
        y = get_dataset_data(dataset, expand_list(i, 8))
        return np.reshape(y, (-1, 8))      
    if j is not None:
        i = get_cel_number(i,j,k)
    return dataset[(i-1)*8:i*8]

def get_cell_coordinates(i,j=None,k=None, can_be_iterable=True):
    dataset = f['SpatialProperties/000000/GRID/NODES']
    if can_be_iterable and is_iterable(i):
        if j is not None:
            i = get_cel_number(i,j,k)
        nodes = get_nodes_index(i)
        y = get_dataset_data(dataset, expand_list(nodes.flatten(), 3))
        return y.reshape((len(i), 8, 3))
    if j is not None:
        i = get_cel_number(i,j,k)
    return [dataset[(n-1)*3:n*3] for n in get_nodes_index(i)]

In [ ]:
size = 2000
i_list = list(np.random.randint(low=1, high=ni, size=size))
j_list = list(np.random.randint(low=1, high=nj, size=size))
k_list = list(np.random.randint(low=1, high=nk, size=size))

cells = get_cel_number(i_list,j_list,k_list)

start_time = datetime.now()
x = get_cell_coordinates(cells)
end_time = datetime.now()
print(f'Elapsed time: {end_time - start_time}')

Elapsed time: 0:00:04.204510


In [ ]:
def time_function(func, size):
    i_list = list(np.random.randint(low=1, high=ni, size=size))
    j_list = list(np.random.randint(low=1, high=nj, size=size))
    k_list = list(np.random.randint(low=1, high=nk, size=size))
    cells = get_cel_number(i_list,j_list,k_list)

    start_time = datetime.now()
    x = func(cells)
    end_time = datetime.now()
    # print(f'{s} elemts: {end_time - start_time} ({(end_time - start_time)/s*1000} per 1000 elemts)')
    return (end_time - start_time)/size*1000 

def time_function_sens(func, arguments_sizes):
    t = []
    for s in arguments_sizes:
        t.append(time_function(func, s))
    return t

def time_function_opt(func, min_val, max_val, fmin_val=None, fmax_val=None):
    if fmin_val is None:
        fmin_val = time_function(func, min_val)
        print(f'{min_val} => {fmin_val}')
    if fmax_val is None:
        fmax_val = time_function(func, max_val)
        print(f'{max_val} => {fmax_val}')
    a_val = int((2*min_val + max_val)/3)
    b_val = int((min_val + 2*max_val)/3)
    if a_val == min_val or b_val == max_val:
        if fmin_val < fmax_val:
            return min_val
        else:
            return max_val
    fa_val = time_function(func, a_val)
    fb_val = time_function(func, b_val)
    print(f'{a_val} => {fa_val}')
    print(f'{b_val} => {fb_val}')
    if fa_val < fb_val:
        return time_function_opt(func, min_val, b_val, fmin_val, fb_val)
    else:
        return time_function_opt(func, a_val, max_val, fa_val, fmax_val)


# t = time_function_sens(get_nodes_index, [100, 200, 500, 1000, 2000, 5000])
    
